## CatBoost-Optunaのサンプルコード

In [1]:
# ライブラリーのインポート
import os

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# ボストンの住宅価格データ
from sklearn.datasets import load_boston

# 前処理
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# CatBoost
import catboost as  cb
from catboost import CatBoost, Pool

# Optuna
import optuna
from sklearn.model_selection import cross_val_score

# 評価指標
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [2]:
# データセットの読込み
boston = load_boston()

# 説明変数の格納
df = pd.DataFrame(boston.data, columns = boston.feature_names)
# 目的変数の追加
df['MEDV'] = boston.target

# データの中身を確認
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


#### 前処理

In [3]:
# ランダムシード値
RANDOM_STATE = 10

# 学習データと評価データの割合
TEST_SIZE = 0.2

# 学習データと評価データを作成
x_train, x_test, y_train, y_test = train_test_split(df.iloc[:, 0:df.shape[1]-1],
                                                    df.iloc[:, df.shape[1]-1],
                                                    test_size=TEST_SIZE,
                                                    random_state=RANDOM_STATE)

In [4]:
def objective(trial):
    
    iterations = trial.suggest_int('iterations', 50, 300)
    depth = trial.suggest_int('depth', 4, 10)                                       
    learning_rate = trial.suggest_loguniform('learning_rate', 0.01, 0.3)               
    random_strength = trial.suggest_int('random_strength', 0, 100)   
    bagging_temperature = trial.suggest_loguniform('bagging_temperature', 0.01, 100.00) 
    od_type = trial.suggest_categorical('od_type', ['IncToDec', 'Iter'])
    od_wait = trial.suggest_int('od_wait', 10, 50)


    model = cb.CatBoostRegressor(iterations = iterations,
                                depth = depth,
                                learning_rate = learning_rate,
                                random_strength = random_strength,
                                bagging_temperature = bagging_temperature,
                                od_type = od_type,
                                od_wait = od_wait
                               )

    score = cross_val_score(model,
                            x_train,
                            y_train,
                            cv=5,
                            scoring="neg_mean_absolute_error"
                           )
    mae = score.mean()

    return mae

In [5]:
%%time
# optunaで最適値を見つける
# 注：cross_val_scoreの出力は全て高いほど良い
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

[I 2021-05-16 02:37:52,956] A new study created in memory with name: no-name-456ccd5f-fc2e-41db-9899-7aedfe8113fc


0:	learn: 8.6374276	total: 62.1ms	remaining: 4.41s
1:	learn: 8.5210440	total: 64.1ms	remaining: 2.24s
2:	learn: 8.4429531	total: 65.9ms	remaining: 1.52s
3:	learn: 8.3318635	total: 68.9ms	remaining: 1.17s
4:	learn: 8.2393306	total: 70.8ms	remaining: 949ms
5:	learn: 8.1435998	total: 72.5ms	remaining: 798ms
6:	learn: 8.0317041	total: 83ms	remaining: 771ms
7:	learn: 7.9249001	total: 84.4ms	remaining: 676ms
8:	learn: 7.7964491	total: 86.4ms	remaining: 605ms
9:	learn: 7.6461657	total: 89.7ms	remaining: 556ms
10:	learn: 7.5218745	total: 94.8ms	remaining: 526ms
11:	learn: 7.4679788	total: 100ms	remaining: 502ms
12:	learn: 7.3726608	total: 102ms	remaining: 463ms
13:	learn: 7.2575991	total: 104ms	remaining: 430ms
14:	learn: 7.1791690	total: 105ms	remaining: 401ms
15:	learn: 7.0716827	total: 107ms	remaining: 375ms
16:	learn: 7.0229334	total: 110ms	remaining: 355ms
17:	learn: 6.9542098	total: 112ms	remaining: 335ms
18:	learn: 6.9116055	total: 114ms	remaining: 318ms
19:	learn: 6.8590174	total: 117m

[I 2021-05-16 02:38:02,991] Trial 0 finished with value: -3.3788407250472714 and parameters: {'iterations': 72, 'depth': 5, 'learning_rate': 0.04378701466670342, 'random_strength': 55, 'bagging_temperature': 15.708883855131923, 'od_type': 'IncToDec', 'od_wait': 12}. Best is trial 0 with value: -3.3788407250472714.


68:	learn: 4.5251607	total: 194ms	remaining: 8.45ms
69:	learn: 4.5140418	total: 196ms	remaining: 5.61ms
70:	learn: 4.4865565	total: 202ms	remaining: 2.85ms
71:	learn: 4.4455706	total: 205ms	remaining: 0us
0:	learn: 8.7203559	total: 47.2ms	remaining: 6.84s
1:	learn: 8.6418447	total: 87.9ms	remaining: 6.33s
2:	learn: 8.5458857	total: 128ms	remaining: 6.09s
3:	learn: 8.4415937	total: 160ms	remaining: 5.67s
4:	learn: 8.3199805	total: 181ms	remaining: 5.09s
5:	learn: 8.2085471	total: 200ms	remaining: 4.67s
6:	learn: 8.1639454	total: 203ms	remaining: 4.02s
7:	learn: 8.0625425	total: 235ms	remaining: 4.04s
8:	learn: 7.9336910	total: 249ms	remaining: 3.79s
9:	learn: 7.8624784	total: 291ms	remaining: 3.96s
10:	learn: 7.7546864	total: 401ms	remaining: 4.92s
11:	learn: 7.6388785	total: 422ms	remaining: 4.71s
12:	learn: 7.5365311	total: 459ms	remaining: 4.7s
13:	learn: 7.5126392	total: 461ms	remaining: 4.34s
14:	learn: 7.4210514	total: 490ms	remaining: 4.28s
15:	learn: 7.3164326	total: 547ms	remai

[I 2021-05-16 02:38:20,777] Trial 1 finished with value: -2.8928640825189915 and parameters: {'iterations': 146, 'depth': 9, 'learning_rate': 0.03325189967779843, 'random_strength': 77, 'bagging_temperature': 9.42099631416822, 'od_type': 'IncToDec', 'od_wait': 11}. Best is trial 1 with value: -2.8928640825189915.


...

0:	learn: 8.7813414	total: 36.5ms	remaining: 4.53s
1:	learn: 8.7429728	total: 64.7ms	remaining: 3.98s
2:	learn: 8.7050383	total: 92.6ms	remaining: 3.77s
3:	learn: 8.6640818	total: 132ms	remaining: 3.99s
4:	learn: 8.6211794	total: 170ms	remaining: 4.07s
5:	learn: 8.5762531	total: 209ms	remaining: 4.15s
6:	learn: 8.5379681	total: 239ms	remaining: 4.03s
7:	learn: 8.5010824	total: 263ms	remaining: 3.85s
8:	learn: 8.4669946	total: 266ms	remaining: 3.43s
9:	learn: 8.4275363	total: 295ms	remaining: 3.39s
10:	learn: 8.4025681	total: 321ms	remaining: 3.33s
11:	learn: 8.3647459	total: 345ms	remaining: 3.25s
12:	learn: 8.3271797	total: 413ms	remaining: 3.56s
13:	learn: 8.2928593	total: 440ms	remaining: 3.49s
14:	learn: 8.2551286	total: 469ms	remaining: 3.44s
15:	learn: 8.2184594	total: 500ms	remaining: 3.4s
16:	learn: 8.1798361	total: 507ms	remaining: 3.22s
17:	learn: 8.1427158	total: 539ms	remaining: 3.2s
18:	learn: 8.1083495	total: 581ms	remaining: 3.24s
19:	learn: 8.0811307	total: 618ms	remain

[I 2021-05-16 02:41:48,829] Trial 8 finished with value: -4.018428514351837 and parameters: {'iterations': 125, 'depth': 10, 'learning_rate': 0.010346308789977146, 'random_strength': 12, 'bagging_temperature': 0.28280439887778097, 'od_type': 'Iter', 'od_wait': 28}. Best is trial 7 with value: -2.0189016584035113.


123:	learn: 5.2713859	total: 3.86s	remaining: 31.1ms
124:	learn: 5.2499642	total: 3.88s	remaining: 0us
0:	learn: 8.4546573	total: 51ms	remaining: 12.2s
1:	learn: 8.1377439	total: 77.4ms	remaining: 9.25s
2:	learn: 7.8323843	total: 110ms	remaining: 8.76s
3:	learn: 7.4467400	total: 137ms	remaining: 8.09s
4:	learn: 7.0911925	total: 164ms	remaining: 7.75s
5:	learn: 6.7444514	total: 193ms	remaining: 7.54s
6:	learn: 6.4514530	total: 237ms	remaining: 7.91s
7:	learn: 6.2113150	total: 268ms	remaining: 7.8s
8:	learn: 6.1244963	total: 272ms	remaining: 7.02s
9:	learn: 5.9124422	total: 300ms	remaining: 6.93s
10:	learn: 5.8128987	total: 333ms	remaining: 6.96s
11:	learn: 5.5787758	total: 359ms	remaining: 6.84s
12:	learn: 5.3767372	total: 421ms	remaining: 7.39s
13:	learn: 5.2138509	total: 450ms	remaining: 7.29s
14:	learn: 5.0969180	total: 483ms	remaining: 7.28s
15:	learn: 4.9592363	total: 518ms	remaining: 7.29s
16:	learn: 4.7940707	total: 522ms	remaining: 6.88s
17:	learn: 4.6586788	total: 549ms	remaini

[I 2021-05-16 02:42:49,530] Trial 9 finished with value: -2.2175817925697023 and parameters: {'iterations': 241, 'depth': 10, 'learning_rate': 0.11828933853491311, 'random_strength': 99, 'bagging_temperature': 57.54687774779825, 'od_type': 'Iter', 'od_wait': 45}. Best is trial 7 with value: -2.0189016584035113.


239:	learn: 0.2368461	total: 9.18s	remaining: 38.3ms
240:	learn: 0.2345006	total: 9.23s	remaining: 0us
CPU times: user 4min 15s, sys: 23 s, total: 4min 38s
Wall time: 4min 56s


In [6]:
# チューニングしたハイパーパラメーターをフィット
optimised_model = cb.CatBoostRegressor(iterations = study.best_params['iterations'],
                                       depth = study.best_params['depth'],
                                       learning_rate = study.best_params['learning_rate'],
                                       random_strength = study.best_params['random_strength'],
                                       bagging_temperature = study.best_params['bagging_temperature'],
                                       od_type = study.best_params['od_type'],
                                       od_wait = study.best_params['od_wait']
                                      )

optimised_model.fit(x_train ,y_train)

# CatBoost推論
y_pred = optimised_model.predict(x_test)

0:	learn: 8.1990532	total: 684us	remaining: 134ms
1:	learn: 7.6643618	total: 4.35ms	remaining: 425ms
2:	learn: 7.2272107	total: 11.2ms	remaining: 725ms
3:	learn: 6.6520430	total: 15.3ms	remaining: 736ms
4:	learn: 6.2663841	total: 20.9ms	remaining: 802ms
5:	learn: 6.0027620	total: 24.4ms	remaining: 776ms
6:	learn: 5.7039948	total: 32.1ms	remaining: 871ms
7:	learn: 5.3652695	total: 39.3ms	remaining: 927ms
8:	learn: 5.0776113	total: 46.6ms	remaining: 973ms
9:	learn: 4.8263310	total: 56.1ms	remaining: 1.05s
10:	learn: 4.6858115	total: 63.1ms	remaining: 1.07s
11:	learn: 4.4901096	total: 72.8ms	remaining: 1.12s
12:	learn: 4.3708827	total: 78.5ms	remaining: 1.11s
13:	learn: 4.2245661	total: 85.8ms	remaining: 1.12s
14:	learn: 4.1181626	total: 88.7ms	remaining: 1.07s
15:	learn: 4.0110823	total: 110ms	remaining: 1.24s
16:	learn: 3.8703520	total: 119ms	remaining: 1.25s
17:	learn: 3.7196390	total: 125ms	remaining: 1.24s
18:	learn: 3.6282867	total: 131ms	remaining: 1.22s
19:	learn: 3.5361535	total:

In [7]:
# 評価
def calculate_scores(true, pred):
    """全ての評価指標を計算する

    Parameters
    ----------
    true (np.array)       : 実測値
    pred (np.array)       : 予測値

    Returns
    -------
    scores (pd.DataFrame) : 各評価指標を纏めた結果

    """
    scores = {}
    scores = pd.DataFrame({'R2': r2_score(true, pred),
                          'MAE': mean_absolute_error(true, pred),
                          'MSE': mean_squared_error(true, pred),
                          'RMSE': np.sqrt(mean_squared_error(true, pred))},
                           index = ['scores'])
    return scores

In [8]:
scores = calculate_scores(y_test, y_pred)
print(scores)

              R2       MAE        MSE      RMSE
scores  0.865739  2.541304  14.041161  3.747154
